In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler
from math import ceil
from itertools import cycle
from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from sklearn.model_selection import train_test_split
# used to provide a value to data that can be then used in equations, eg. Friday = 1 etc.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import style
style.use('ggplot')
from googletrans import Translator

from xgboost import XGBRegressor
from xgboost import plot_importance

import time
import sys
import gc
import pickle

In [2]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

In [14]:
#Read in .csv files into pandas dataframes
train       = pd.read_csv('sales_train.csv')
test        = pd.read_csv('test.csv').set_index('ID')
submission  = pd.read_csv('sample_submission.csv')
items       = pd.read_csv('items.csv')
item_cats   = pd.read_csv('item_categories.csv')
shops       = pd.read_csv('shops.csv')
shops_t     = pd.read_csv('shops_translated.csv')
items_t     = pd.read_csv('items_translated.csv')

In [15]:
#Down casts the data entries from int64 to int32 and float64 to float32
#This reduces the size of the records by almost half. (From 134mb to 61mb)
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [16]:
#Calls the downcasting function
train      = downcast_dtypes(train)
test       = downcast_dtypes(test)
submission = downcast_dtypes(submission)
items      = downcast_dtypes(items)
items_t    = downcast_dtypes(items_t)
item_cats  = downcast_dtypes(item_cats)
shops      = downcast_dtypes(shops)
shops_t    = downcast_dtypes(shops_t)

In [17]:
#Adds item_category_id to train dataframe and drops the item_name column
#Formats the date from string to datetime object
train = train.merge(items, on='item_id')
train = train.drop(columns = 'item_name')
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')

In [18]:
#Removes outlier price items from train dataframe
train = train[train.item_price < 90000]
train = train[train.item_cnt_day < 999]

#replaces the one instance of a negative price item 
#with the median item_price of the item with the id of 2973 and in shop id 32
median = train[(train.shop_id==32)
               &(train.item_id==2973)
               &(train.date_block_num==4)
               &(train.item_price>0)].item_price.median()

In [20]:
#Moves item_cnt_day column to the last column
train_cnt = train['item_cnt_day']
train.drop(labels=['item_cnt_day'], axis=1, inplace = True)
train.insert(6, 'item_cnt_day', train_cnt)

In [22]:
#Sums the item_cnt_day by date_block_month
#Drops the date column
#resets the index to start at 0 again
train_grouped_month = pd.DataFrame(train.groupby(['date_block_num', 'shop_id', 
                                                  'item_category_id', 'item_id',
                                                  'item_price'])['item_cnt_day'].sum().reset_index())

In [25]:
#renames item_cnt_day to item_cnt_month
train_grouped_month.rename(columns = {'item_cnt_day':'item_cnt_month'}, inplace = True)

In [26]:
train_grouped_month

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_month
0,0,0,2,5572,1322.0,10.0
1,0,0,2,5573,560.0,1.0
2,0,0,2,5575,806.0,4.0
3,0,0,2,5576,2231.0,5.0
4,0,0,2,5609,2381.0,1.0
...,...,...,...,...,...,...
1739014,33,59,79,17717,1250.0,4.0
1739015,33,59,79,17717,1999.0,1.0
1739016,33,59,83,22087,119.0,6.0
1739017,33,59,83,22088,119.0,2.0


In [27]:
#Converts all categorical features into strings because
#nominal intergers can not be converted to binary encoding
train_grouped_month['date_block_num']   = [('month ' + str(i)) for i in train_grouped_month['date_block_num']]
train_grouped_month['shop_id']          = [('shop ' + str(i)) for i in train_grouped_month['shop_id']]
train_grouped_month['item_category_id'] = [('item_category ' + str(i)) for i in train_grouped_month['item_category_id']]
train_grouped_month['item_id']          = [('item ' + str(i)) for i in train_grouped_month['item_id']]

In [28]:
train_grouped_month

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_month
0,month 0,shop 0,item_category 2,item 5572,1322.0,10.0
1,month 0,shop 0,item_category 2,item 5573,560.0,1.0
2,month 0,shop 0,item_category 2,item 5575,806.0,4.0
3,month 0,shop 0,item_category 2,item 5576,2231.0,5.0
4,month 0,shop 0,item_category 2,item 5609,2381.0,1.0
...,...,...,...,...,...,...
1739014,month 33,shop 59,item_category 79,item 17717,1250.0,4.0
1739015,month 33,shop 59,item_category 79,item 17717,1999.0,1.0
1739016,month 33,shop 59,item_category 83,item 22087,119.0,6.0
1739017,month 33,shop 59,item_category 83,item 22088,119.0,2.0
